In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from sklearn.model_selection import train_test_split
%load_ext tensorboard

In [2]:
import glob
files = glob.glob("../../datasets/xalan-2.5.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [3]:
ant_master

,name,version,name.1,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,xalan,2.5.0,org.apache.xpath.res.XPATHErrorResources_sv,3,4,0,1,4,1,0,...,0.000000,0,0.976190,1.000000,0,0,618.333333,1,0.3333,1
1,xalan,2.5.0,org.apache.xalan.xsltc.compiler.TransletOutput,5,3,0,20,33,0,0,...,1.000000,1,0.933333,0.333333,2,7,33.800000,6,1.8000,0
2,xalan,2.5.0,org.apache.xalan.lib.Extensions,13,1,0,11,54,76,1,...,0.500000,1,0.000000,0.208333,0,0,27.230769,5,1.9231,1
3,xalan,2.5.0,org.apache.xml.serializer.ToHTMLStream,28,3,0,12,104,222,1,...,0.888889,5,0.866667,0.196296,2,26,87.250000,3,1.1429,4
4,xalan,2.5.0,org.apache.xpath.functions.FuncConcat,4,7,0,7,13,6,0,...,0.000000,0,0.960000,0.500000,3,8,19.250000,1,0.7500,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
798,xalan,2.5.0,org.apache.xalan.xsltc.dom.DOMBuilder,0,1,0,5,0,0,4,...,0.000000,0,0.000000,0.000000,0,0,0.000000,0,0.0000,0
799,xalan,2.5.0,org.apache.xalan.processor.ProcessorLRE,4,5,0,17,77,6,1,...,0.000000,0,0.986111,0.500000,4,6,117.000000,3,1.2500,1
800,xalan,2.5.0,org.apache.xpath.axes.FilterExprIterator,14,6,0,11,32,0,1,...,1.000000,2,0.914286,0.234694,4,8,10.357143,3,1.2143,0
801,xalan,2.5.0,org.apache.xpath.res.XPATHMessages,5,2,1,2,13,4,1,...,1.000000,0,0.666667,0.500000,0,0,21.600000,7,2.6000,1


In [4]:
train, test = train_test_split(ant_master, test_size=0.2)

In [5]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [6]:
x_train

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
647,56,2,2,25,84,714,18,7,56,0.739394,839,0.833333,1,0.363636,0.160714,1,4,13.875000,9,1.5714
333,4,2,1,5,11,4,1,5,4,2.000000,63,0.000000,0,0.823529,0.500000,1,4,14.750000,5,1.7500
354,7,4,0,22,49,21,4,19,5,2.000000,452,0.000000,0,0.923077,0.339286,2,7,63.571429,14,5.1429
111,2,1,0,1,23,1,1,0,2,2.000000,145,0.000000,0,0.000000,0.500000,0,0,71.500000,8,4.0000
763,2,1,0,10,14,0,2,8,2,0.000000,104,1.000000,0,0.000000,0.583333,0,0,50.500000,1,0.5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,39,1,0,1,56,469,1,0,39,0.578947,198,0.000000,0,0.000000,0.256410,0,0,4.051282,1,0.9744
738,6,3,0,13,28,0,0,13,6,0.000000,101,1.000000,1,0.934211,0.305556,2,8,15.666667,1,0.8333
217,34,3,0,14,143,369,5,13,24,0.903263,1703,0.576923,0,0.384615,0.142857,1,2,48.323529,17,2.0882
163,9,1,0,7,29,12,5,2,8,0.625000,581,1.000000,0,0.000000,0.416667,0,0,63.222222,7,2.5556


In [7]:
x_test

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
480,22,2,0,42,54,199,13,32,20,0.571429,466,1.000000,0,0.622642,0.215909,2,4,20.136364,7,1.4091
568,10,6,0,30,48,1,2,28,10,0.518519,302,1.000000,2,0.925234,0.242857,4,5,28.900000,8,1.6000
472,2,4,0,6,9,1,1,5,1,2.000000,27,0.000000,0,0.995261,0.625000,2,2,12.500000,1,0.5000
9,74,3,2,47,119,2539,37,17,70,0.966051,1136,0.913043,4,0.728625,0.073443,3,19,14.040541,16,1.8378
627,6,3,0,20,37,15,10,11,5,1.133333,273,1.000000,0,0.897436,0.560000,1,4,44.000000,16,3.1667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,3,2,0,3,5,3,1,2,0,2.000000,24,0.000000,0,0.500000,0.555556,0,0,7.000000,2,1.3333
710,5,1,0,6,27,10,2,4,5,2.000000,149,0.000000,0,0.000000,0.300000,0,0,28.800000,1,0.8000
135,25,1,0,3,71,254,2,1,20,0.892857,582,1.000000,1,0.000000,0.152727,0,0,22.000000,3,1.0800
360,14,4,0,9,40,0,1,8,13,0.076923,174,1.000000,1,0.941704,0.295918,2,3,11.357143,1,0.9286


In [8]:
y_train = train['bug']
y_test = test['bug']

In [9]:
y_train

647    1
333    0
354    1
111    2
763    0
      ..
117    1
738    0
217    5
163    1
592    1
Name: bug, Length: 642, dtype: int64

In [10]:
y_test

480    0
568    0
472    2
9      2
627    0
      ..
260    0
710    3
135    0
360    0
49     2
Name: bug, Length: 161, dtype: int64

In [11]:
x_train_np = np.array(x_train)
x_test_np = np.array(x_test)

In [12]:
x_train_np

array([[56.        ,  2.        ,  2.        , ..., 13.875     ,
         9.        ,  1.5714    ],
       [ 4.        ,  2.        ,  1.        , ..., 14.75      ,
         5.        ,  1.75      ],
       [ 7.        ,  4.        ,  0.        , ..., 63.57142857,
        14.        ,  5.1429    ],
       ...,
       [34.        ,  3.        ,  0.        , ..., 48.32352941,
        17.        ,  2.0882    ],
       [ 9.        ,  1.        ,  0.        , ..., 63.22222222,
         7.        ,  2.5556    ],
       [ 5.        ,  2.        ,  0.        , ..., 20.4       ,
         4.        ,  1.8       ]])

In [13]:
x_test_np

array([[22.        ,  2.        ,  0.        , ..., 20.13636364,
         7.        ,  1.4091    ],
       [10.        ,  6.        ,  0.        , ..., 28.9       ,
         8.        ,  1.6       ],
       [ 2.        ,  4.        ,  0.        , ..., 12.5       ,
         1.        ,  0.5       ],
       ...,
       [25.        ,  1.        ,  0.        , ..., 22.        ,
         3.        ,  1.08      ],
       [14.        ,  4.        ,  0.        , ..., 11.35714286,
         1.        ,  0.9286    ],
       [ 3.        ,  4.        ,  0.        , ..., 49.66666667,
         8.        ,  3.        ]])

In [14]:
train_x = np.reshape(x_train_np, (x_train_np.shape[0], 1, x_train_np.shape[1]))
test_x = np.reshape(x_test_np, (x_test_np.shape[0], 1, x_test_np.shape[1]))

In [15]:
train_y = y_train.to_numpy()
test_y = y_test.to_numpy()

In [16]:
train_x

array([[[56.        ,  2.        ,  2.        , ..., 13.875     ,
          9.        ,  1.5714    ]],

       [[ 4.        ,  2.        ,  1.        , ..., 14.75      ,
          5.        ,  1.75      ]],

       [[ 7.        ,  4.        ,  0.        , ..., 63.57142857,
         14.        ,  5.1429    ]],

       ...,

       [[34.        ,  3.        ,  0.        , ..., 48.32352941,
         17.        ,  2.0882    ]],

       [[ 9.        ,  1.        ,  0.        , ..., 63.22222222,
          7.        ,  2.5556    ]],

       [[ 5.        ,  2.        ,  0.        , ..., 20.4       ,
          4.        ,  1.8       ]]])

In [17]:
test_y

array([0, 0, 2, 2, 0, 1, 0, 0, 1, 0, 0, 1, 2, 0, 0, 1, 2, 0, 0, 0, 0, 1,
       2, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 2, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 2, 0, 0,
       1, 1, 0, 0, 1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 2, 0, 2, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1,
       1, 0, 2, 1, 2, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       2, 1, 0, 0, 0, 2, 1, 0, 2, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1,
       2, 0, 0, 3, 0, 0, 2])

In [18]:
# Designing and initializing the model.
from keras.layers import LSTM, Dense, SimpleRNN
# Designing and initializing the model.
model = Sequential()
model.add(LSTM(200, return_sequences=True))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(50, dropout = 0.2, return_sequences=True))
model.add(LSTM(20, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = keras.optimizers.Adam(learning_rate=0.001) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [19]:
# Fitting the model on training data.
import datetime
log_dir = '/home/bavanya/Desktop/tensorboard/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(train_x, train_y, epochs = 100, callbacks=[tensorboard_callback])

Epoch 1/100
21/21 [==============================] - 4s 17ms/step - loss: 1.1994 - mse: 1.1994 - mae: 0.6880 - root_mean_squared_error: 1.0952 - mean_squared_logarithmic_error: 0.2748
Epoch 2/100
21/21 [==============================] - 0s 5ms/step - loss: 0.9257 - mse: 0.9257 - mae: 0.6542 - root_mean_squared_error: 0.9621 - mean_squared_logarithmic_error: 0.2184
Epoch 3/100
21/21 [==============================] - 0s 5ms/step - loss: 0.8705 - mse: 0.8705 - mae: 0.6274 - root_mean_squared_error: 0.9330 - mean_squared_logarithmic_error: 0.1861
Epoch 4/100
21/21 [==============================] - 0s 5ms/step - loss: 0.8150 - mse: 0.8150 - mae: 0.6091 - root_mean_squared_error: 0.9028 - mean_squared_logarithmic_error: 0.1816
Epoch 5/100
21/21 [==============================] - 0s 5ms/step - loss: 0.7555 - mse: 0.7555 - mae: 0.6033 - root_mean_squared_error: 0.8692 - mean_squared_logarithmic_error: 0.1738
Epoch 6/100
21/21 [==============================] - 0s 5ms/step - loss: 0.7552 - ms

21/21 [==============================] - 0s 5ms/step - loss: 0.5843 - mse: 0.5843 - mae: 0.5068 - root_mean_squared_error: 0.7644 - mean_squared_logarithmic_error: 0.1309
Epoch 45/100
21/21 [==============================] - 0s 6ms/step - loss: 0.5798 - mse: 0.5798 - mae: 0.5013 - root_mean_squared_error: 0.7614 - mean_squared_logarithmic_error: 0.1311
Epoch 46/100
21/21 [==============================] - 0s 7ms/step - loss: 0.5990 - mse: 0.5990 - mae: 0.5045 - root_mean_squared_error: 0.7739 - mean_squared_logarithmic_error: 0.1325
Epoch 47/100
21/21 [==============================] - 0s 5ms/step - loss: 0.8061 - mse: 0.8061 - mae: 0.5783 - root_mean_squared_error: 0.8978 - mean_squared_logarithmic_error: 0.1678
Epoch 48/100
21/21 [==============================] - 0s 5ms/step - loss: 0.6081 - mse: 0.6081 - mae: 0.5196 - root_mean_squared_error: 0.7798 - mean_squared_logarithmic_error: 0.1411
Epoch 49/100
21/21 [==============================] - 0s 6ms/step - loss: 0.6352 - mse: 0.635

21/21 [==============================] - 0s 5ms/step - loss: 0.5868 - mse: 0.5868 - mae: 0.4732 - root_mean_squared_error: 0.7660 - mean_squared_logarithmic_error: 0.1225
Epoch 89/100
21/21 [==============================] - 0s 5ms/step - loss: 0.5743 - mse: 0.5743 - mae: 0.4719 - root_mean_squared_error: 0.7578 - mean_squared_logarithmic_error: 0.1219
Epoch 90/100
21/21 [==============================] - 0s 5ms/step - loss: 0.5164 - mse: 0.5164 - mae: 0.4634 - root_mean_squared_error: 0.7186 - mean_squared_logarithmic_error: 0.1190
Epoch 91/100
21/21 [==============================] - 0s 5ms/step - loss: 0.5597 - mse: 0.5597 - mae: 0.4599 - root_mean_squared_error: 0.7481 - mean_squared_logarithmic_error: 0.1202
Epoch 92/100
21/21 [==============================] - 0s 5ms/step - loss: 0.5711 - mse: 0.5711 - mae: 0.4683 - root_mean_squared_error: 0.7557 - mean_squared_logarithmic_error: 0.1209
Epoch 93/100
21/21 [==============================] - 0s 5ms/step - loss: 0.5368 - mse: 0.536

In [20]:
%tensorboard --logdir '/home/bavanya/Desktop/tensorboard/'

Reusing TensorBoard on port 6006 (pid 3750), started 3:20:05 ago. (Use '!kill 3750' to kill it.)

In [21]:
score = model.evaluate(test_x, test_y)
score

6/6 [==============================] - 1s 2ms/step - loss: 0.6486 - mse: 0.6486 - mae: 0.5895 - root_mean_squared_error: 0.8054 - mean_squared_logarithmic_error: 0.2199


[0.6486241817474365,
 0.6486241817474365,
 0.5894823670387268,
 0.8053720593452454,
 0.21994969248771667]